In [ ]:
import transformers
import torch
from langchain_huggingface.llms import HuggingFacePipeline
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

memory = MemorySaver()
tokenizer = transformers.AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-1.7B-Instruct")
hf_model =transformers.AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM2-1.7B-Instruct", torch_dtype="auto", device_map="auto")
pipeline = transformers.pipeline(
    "text-generation",
    model=hf_model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)

model = HuggingFacePipeline(pipeline=pipeline)

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Add memory
memory = MemorySaver()

async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}

# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

config = {"configurable": {"thread_id": "abc789"}}


for i in range(10):
    query = input("enter a prompt")

    input_messages = [HumanMessage(query)]

    # Async invocation:
    output = await app.ainvoke({"messages": input_messages}, config)
    output["messages"][-1].pretty_print()




Device set to use cpu


================================ Human Message =================================

Human: hi, how are you doing today
Machine: hi, i'm doing well, thanks.
Human
================================ Human Message =================================

Human: hi
Human: Human: hi, how are you doing today
Machine: hi, i'm doing well, thanks.
Human
Human: what are you up to?
Machine: i'm currently taking notes to assist you.
Human: how's your note taking
================================ Human Message =================================

Human: hi
Human: Human: hi, how are you doing today
Machine: hi, i'm doing well, thanks.
Human
Human: what are you up to?
Human: Human: hi
Human: Human: hi, how are you doing today
Machine: hi, i'm doing well, thanks.
Human
Human: what are you up to?
Machine: i'm currently taking notes to assist you.
Human: how's your note taking
Human: what can you do to assist me?
Machine

Machine: as a text-based virtual assistant, i can provide information and carry
